# LoanTap Loan Default Prediction using Logistic Regression

**Author:** Vidyasagar | Data Scientist  
**Date:** February 2026  
**Project:** Predictive Analytics in Asymmetric Data — A Robust Classification Framework

---

## Problem Statement

LoanTap is an online platform committed to delivering customized loan products to millennials. They innovate in an otherwise dull loan segment, to deliver instant, flexible loans on consumer-friendly terms to salaried professionals and businessmen.

The key business problem is to **predict whether a borrower will default on their loan** (Charged Off) or **fully pay** it back. This is critical because:

1. **False Positives (predicting default when they won't):** The company loses business opportunity and potential interest income.
2. **False Negatives (predicting fully paid when they will default):** The company faces Non-Performing Assets (NPA), leading to financial losses.

We will build a **Logistic Regression** model to predict the `loan_status` (target variable) using various borrower attributes and loan characteristics.

### Approach
1. Exploratory Data Analysis (EDA)
2. Feature Engineering
3. Data Preprocessing (Missing values, Outliers, Scaling)
4. Logistic Regression Model Building
5. Model Evaluation (ROC-AUC, Precision-Recall, Classification Report)
6. Precision vs Recall Tradeoff Analysis
7. Actionable Insights & Recommendations

---
## 1. Import Libraries & Load Dataset

In [ ]:
# ============================================================
# Import Required Libraries
# ============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, precision_recall_curve, auc, accuracy_score,
    precision_score, recall_score, f1_score
)
import statsmodels.api as sm

# Plot settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
sns.set_style('whitegrid')
sns.set_palette('husl')

print("All libraries imported successfully!")

In [ ]:
# ============================================================
# Load the Dataset
# ============================================================
df = pd.read_csv('../data/LoanTapData.csv')
print(f"Dataset Shape: {df.shape}")
print(f"Number of Records: {df.shape[0]}")
print(f"Number of Features: {df.shape[1]}")
df.head()

---
## 2. Exploratory Data Analysis (EDA)

### 2.1 Data Structure & Characteristics

In [ ]:
# ============================================================
# 2.1.1 Dataset Info & Data Types
# ============================================================
print("=" * 60)
print("DATASET INFORMATION")
print("=" * 60)
df.info()

In [ ]:
# ============================================================
# 2.1.2 Statistical Summary of Numerical Columns
# ============================================================
print("=" * 60)
print("STATISTICAL SUMMARY - NUMERICAL FEATURES")
print("=" * 60)
df.describe().T.round(2)

In [ ]:
# ============================================================
# 2.1.3 Statistical Summary of Categorical Columns
# ============================================================
print("=" * 60)
print("STATISTICAL SUMMARY - CATEGORICAL FEATURES")
print("=" * 60)
df.describe(include='object').T

In [ ]:
# ============================================================
# 2.1.4 Missing Values Analysis
# ============================================================
print("=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct.round(2)})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing %', ascending=False)
print(missing_df)
print(f"\nTotal columns with missing values: {len(missing_df)}")

# Visualize missing values
fig, ax = plt.subplots(figsize=(10, 5))
missing_df['Missing %'].plot(kind='barh', color=sns.color_palette('RdYlGn_r', len(missing_df)), ax=ax)
ax.set_title('Missing Values Percentage by Feature', fontsize=16, fontweight='bold')
ax.set_xlabel('Missing Percentage (%)')
for i, v in enumerate(missing_df['Missing %']):
    ax.text(v + 0.1, i, f'{v:.2f}%', va='center', fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/missing_values.png', dpi=300, bbox_inches='tight')
plt.show()
print("\n>> Insight: mort_acc has the highest missing percentage (~8%). emp_title (~4%) and emp_length (~3%) also have notable missing values.")

In [ ]:
# ============================================================
# 2.1.5 Duplicate Values Check
# ============================================================
print("=" * 60)
print("DUPLICATE VALUES CHECK")
print("=" * 60)
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
print(f"Percentage of duplicates: {(duplicates/len(df))*100:.2f}%")
if duplicates > 0:
    df.drop_duplicates(inplace=True)
    print(f"Duplicates removed. New shape: {df.shape}")

### 2.2 Target Variable Analysis

In [ ]:
# ============================================================
# 2.2.1 Target Variable Distribution
# ============================================================
print("=" * 60)
print("TARGET VARIABLE - LOAN STATUS DISTRIBUTION")
print("=" * 60)
target_counts = df['loan_status'].value_counts()
target_pct = df['loan_status'].value_counts(normalize=True) * 100

print(f"\nFully Paid: {target_counts['Fully Paid']} ({target_pct['Fully Paid']:.2f}%)")
print(f"Charged Off: {target_counts['Charged Off']} ({target_pct['Charged Off']:.2f}%)")

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Count Plot
colors = ['#2ecc71', '#e74c3c']
sns.countplot(data=df, x='loan_status', palette=colors, ax=axes[0], edgecolor='black')
axes[0].set_title('Loan Status Distribution', fontsize=16, fontweight='bold')
axes[0].set_xlabel('Loan Status')
axes[0].set_ylabel('Count')
for p in axes[0].patches:
    axes[0].annotate(f'{int(p.get_height())}\n({p.get_height()/len(df)*100:.1f}%)', 
                     (p.get_x() + p.get_width()/2., p.get_height()),
                     ha='center', va='bottom', fontweight='bold', fontsize=13)

# Pie Chart
axes[1].pie(target_counts, labels=target_counts.index, autopct='%1.1f%%',
            colors=colors, startangle=90, explode=(0.05, 0.05),
            shadow=True, textprops={'fontsize': 13, 'fontweight': 'bold'})
axes[1].set_title('Loan Status Proportion', fontsize=16, fontweight='bold')

plt.tight_layout()
plt.savefig('../figures/target_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n>> ANSWER Q1: {target_pct['Fully Paid']:.2f}% of customers have fully paid their loan amount.")
print(">> Insight: The dataset is imbalanced with ~76% Fully Paid and ~24% Charged Off loans.")

### 2.3 Univariate Analysis - Continuous Variables

In [ ]:
# ============================================================
# 2.3.1 Distribution of Continuous Variables
# ============================================================
continuous_cols = ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti', 
                   'open_acc', 'revol_bal', 'revol_util', 'total_acc']

fig, axes = plt.subplots(3, 3, figsize=(20, 18))
axes = axes.flatten()

for idx, col in enumerate(continuous_cols):
    ax = axes[idx]
    sns.histplot(df[col].dropna(), kde=True, bins=40, color=sns.color_palette('husl', 9)[idx], ax=ax, edgecolor='black', alpha=0.7)
    ax.axvline(df[col].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df[col].mean():.2f}')
    ax.axvline(df[col].median(), color='blue', linestyle='-.', linewidth=2, label=f'Median: {df[col].median():.2f}')
    ax.set_title(f'Distribution of {col}', fontsize=14, fontweight='bold')
    ax.legend(fontsize=9)

plt.suptitle('Univariate Analysis - Distribution of Continuous Variables', fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../figures/univariate_continuous.png', dpi=300, bbox_inches='tight')
plt.show()

print(">> Insights:")
print("   - loan_amnt: Right-skewed distribution with most loans between $5,000 - $20,000")
print("   - int_rate: Approximately normal distribution centered around 12-13%")
print("   - annual_inc: Heavily right-skewed with most incomes between $30,000 - $100,000")
print("   - dti: Right-skewed; most borrowers have DTI between 10-25")
print("   - revol_bal: Highly right-skewed indicating few borrowers with very high revolving balances")

In [ ]:
# ============================================================
# 2.3.2 Box Plots for Outlier Detection
# ============================================================
fig, axes = plt.subplots(3, 3, figsize=(20, 18))
axes = axes.flatten()

for idx, col in enumerate(continuous_cols):
    ax = axes[idx]
    sns.boxplot(data=df, y=col, ax=ax, color=sns.color_palette('Set2')[idx % 8], width=0.5)
    
    # Calculate IQR
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)).sum()
    
    ax.set_title(f'{col}\n(Outliers: {outliers})', fontsize=13, fontweight='bold')

plt.suptitle('Box Plots - Outlier Detection in Continuous Variables', fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../figures/boxplots_outliers.png', dpi=300, bbox_inches='tight')
plt.show()

print(">> Insights on Outliers:")
print("   - annual_inc has significant outliers (very high income earners)")
print("   - revol_bal shows extreme outliers on the higher end")
print("   - open_acc and total_acc have moderate outliers")
print("   - These outliers need treatment before model building")

### 2.4 Univariate Analysis - Categorical Variables

In [ ]:
# ============================================================
# 2.4.1 Categorical Variable Distributions
# ============================================================
cat_cols = ['term', 'grade', 'home_ownership', 'verification_status', 'purpose', 'initial_list_status', 'application_type']

fig, axes = plt.subplots(4, 2, figsize=(18, 24))
axes = axes.flatten()

for idx, col in enumerate(cat_cols):
    ax = axes[idx]
    order = df[col].value_counts().index
    sns.countplot(data=df, x=col, order=order, palette='viridis', ax=ax, edgecolor='black')
    ax.set_title(f'Distribution of {col}', fontsize=14, fontweight='bold')
    ax.tick_params(axis='x', rotation=45)
    for p in ax.patches:
        ax.annotate(f'{int(p.get_height())}', 
                     (p.get_x() + p.get_width()/2., p.get_height()),
                     ha='center', va='bottom', fontsize=10, fontweight='bold')

# Remove empty subplot
axes[-1].set_visible(False)

plt.suptitle('Univariate Analysis - Categorical Variables', fontsize=18, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('../figures/univariate_categorical.png', dpi=300, bbox_inches='tight')
plt.show()

print(">> Insights:")
print(f"   - ANSWER Q3: The majority of people have home ownership as '{df['home_ownership'].value_counts().index[0]}'")
print("   - 36-month term loans are significantly more popular than 60-month")
print("   - Grades B and C are the most common loan grades")
print("   - debt_consolidation is the dominant purpose for loans")
print("   - Individual applications far outnumber joint applications")

In [ ]:
# ============================================================
# 2.4.2 Employment Title Analysis (Top 20)
# ============================================================
fig, ax = plt.subplots(figsize=(14, 8))
top_titles = df['emp_title'].value_counts().head(20)
top_titles.plot(kind='barh', color=sns.color_palette('coolwarm', 20), ax=ax, edgecolor='black')
ax.set_title('Top 20 Employment Titles', fontsize=16, fontweight='bold')
ax.set_xlabel('Count')
ax.set_ylabel('Job Title')
for i, v in enumerate(top_titles):
    ax.text(v + 5, i, str(v), va='center', fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/top_emp_titles.png', dpi=300, bbox_inches='tight')
plt.show()

top2 = df['emp_title'].value_counts().head(2).index.tolist()
print(f"\n>> ANSWER Q5: The top 2 most afforded job titles are: '{top2[0]}' and '{top2[1]}'")

### 2.5 Bivariate Analysis

In [ ]:
# ============================================================
# 2.5.1 Loan Status vs Numerical Features
# ============================================================
fig, axes = plt.subplots(3, 3, figsize=(20, 18))
axes = axes.flatten()

for idx, col in enumerate(continuous_cols):
    ax = axes[idx]
    sns.boxplot(data=df, x='loan_status', y=col, palette=['#2ecc71', '#e74c3c'], ax=ax)
    ax.set_title(f'Loan Status vs {col}', fontsize=13, fontweight='bold')

plt.suptitle('Bivariate Analysis - Loan Status vs Numerical Features', fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../figures/bivariate_numerical.png', dpi=300, bbox_inches='tight')
plt.show()

print(">> Insights:")
print("   - Charged Off loans tend to have higher interest rates")
print("   - Higher loan amounts are slightly more associated with defaults")
print("   - DTI ratio is slightly higher for charged off loans")
print("   - Annual income doesn't show a dramatic difference between groups")

In [ ]:
# ============================================================
# 2.5.2 Loan Status vs Categorical Features
# ============================================================
fig, axes = plt.subplots(3, 2, figsize=(18, 20))
axes = axes.flatten()

cat_bi_cols = ['grade', 'term', 'home_ownership', 'verification_status', 'purpose', 'emp_length']

for idx, col in enumerate(cat_bi_cols):
    ax = axes[idx]
    ct = pd.crosstab(df[col], df['loan_status'], normalize='index') * 100
    ct.plot(kind='bar', stacked=True, color=['#e74c3c', '#2ecc71'], ax=ax, edgecolor='black')
    ax.set_title(f'Loan Status by {col}', fontsize=14, fontweight='bold')
    ax.set_ylabel('Percentage (%)')
    ax.tick_params(axis='x', rotation=45)
    ax.legend(title='Loan Status', loc='upper right')

plt.suptitle('Bivariate Analysis - Loan Status vs Categorical Features', fontsize=18, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('../figures/bivariate_categorical.png', dpi=300, bbox_inches='tight')
plt.show()

# Check Grade A performance
grade_a_fullypaid = pd.crosstab(df['grade'], df['loan_status'], normalize='index')['Fully Paid']['A'] * 100
print(f"\n>> ANSWER Q4: People with grade 'A' fully pay {grade_a_fullypaid:.1f}% of the time.")
print(f"   People with grades 'A' are more likely to fully pay their loan = TRUE")
print("\n>> Insights:")
print("   - Clear gradient: Higher grade (A) = lower default rate, Lower grade (G) = higher default rate")
print("   - 60-month term loans have higher default rates than 36-month")
print("   - Home ownership category doesn't dramatically affect default rates")
print("   - Small business loans have the highest default rates among purposes")

In [ ]:
# ============================================================
# 2.5.3 Correlation Analysis
# ============================================================
numerical_df = df[continuous_cols + ['pub_rec', 'mort_acc', 'pub_rec_bankruptcies']].copy()

fig, ax = plt.subplots(figsize=(14, 10))
corr_matrix = numerical_df.corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdYlBu_r',
            center=0, square=True, linewidths=1, ax=ax,
            cbar_kws={'shrink': 0.8})
ax.set_title('Correlation Heatmap - Numerical Features', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

# Loan Amount vs Installment correlation
corr_loan_inst = df['loan_amnt'].corr(df['installment'])
print(f"\n>> ANSWER Q2: Correlation between Loan Amount and Installment = {corr_loan_inst:.4f}")
print(f"   There is a very strong positive correlation between loan_amnt and installment.")
print(f"   This makes sense because installment is directly calculated from the loan amount.")
print("\n>> Other Insights:")
print("   - loan_amnt and installment are highly correlated (multicollinearity concern)")
print("   - open_acc and total_acc show moderate positive correlation")
print("   - pub_rec and pub_rec_bankruptcies are positively correlated")

In [ ]:
# ============================================================
# 2.5.4 Scatter Plot: Loan Amount vs Installment
# ============================================================
fig, ax = plt.subplots(figsize=(12, 8))
colors_map = {'Fully Paid': '#2ecc71', 'Charged Off': '#e74c3c'}
for status in ['Fully Paid', 'Charged Off']:
    mask = df['loan_status'] == status
    ax.scatter(df.loc[mask, 'loan_amnt'], df.loc[mask, 'installment'], 
               alpha=0.3, s=10, c=colors_map[status], label=status)
ax.set_title('Loan Amount vs Installment (colored by Loan Status)', fontsize=16, fontweight='bold')
ax.set_xlabel('Loan Amount ($)')
ax.set_ylabel('Installment ($)')
ax.legend(markerscale=5)
plt.tight_layout()
plt.savefig('../figures/scatter_loan_installment.png', dpi=300, bbox_inches='tight')
plt.show()

print(">> Insight: The strong linear relationship between loan_amnt and installment is clearly visible.")
print("   Two distinct bands correspond to 36-month and 60-month terms.")

---
## 3. Data Preprocessing

### 3.1 Feature Engineering

In [ ]:
# ============================================================
# 3.1.1 Create a working copy
# ============================================================
df_processed = df.copy()
print(f"Working dataset shape: {df_processed.shape}")

In [ ]:
# ============================================================
# 3.1.2 Feature Engineering - Create Flag Variables
# ============================================================

# Flag: pub_rec > 0 then 1 else 0
df_processed['pub_rec_flag'] = (df_processed['pub_rec'] > 0).astype(int)

# Flag: mort_acc > 0 then 1 else 0  
df_processed['mort_acc_flag'] = (df_processed['mort_acc'] > 0).astype(int)

# Flag: pub_rec_bankruptcies > 0 then 1 else 0
df_processed['pub_rec_bankruptcies_flag'] = (df_processed['pub_rec_bankruptcies'] > 0).astype(int)

print("Flag variables created:")
print(f"  pub_rec_flag distribution: \n{df_processed['pub_rec_flag'].value_counts()}")
print(f"\n  mort_acc_flag distribution: \n{df_processed['mort_acc_flag'].value_counts()}")
print(f"\n  pub_rec_bankruptcies_flag distribution: \n{df_processed['pub_rec_bankruptcies_flag'].value_counts()}")

print("\n>> Insight: These binary flags capture whether a borrower has ANY public records, mortgages, or bankruptcies.")

In [ ]:
# ============================================================
# 3.1.3 Feature Engineering - Extract State from Address
# ============================================================
df_processed['state'] = df_processed['Address'].apply(lambda x: str(x).split('\n')[-1].split()[0] if pd.notna(x) else np.nan)
print(f"Unique states: {df_processed['state'].nunique()}")
print(f"\nTop 10 states:")
print(df_processed['state'].value_counts().head(10))

In [ ]:
# ============================================================
# 2.5.5 Geographic Analysis - Default Rates by State
# ============================================================
state_default = df_processed.groupby('state')['loan_status'].apply(
    lambda x: (x == 'Charged Off').mean() * 100
).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(16, 8))
state_default.plot(kind='bar', color=sns.color_palette('RdYlGn_r', len(state_default)), ax=ax, edgecolor='black')
ax.set_title('Default Rate by State', fontsize=16, fontweight='bold')
ax.set_ylabel('Default Rate (%)')
ax.set_xlabel('State')
ax.axhline(y=state_default.mean(), color='red', linestyle='--', linewidth=2, label=f'Average: {state_default.mean():.1f}%')
ax.legend(fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../figures/default_by_state.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n>> ANSWER Q9: Will the results be affected by geographical location? YES")
print(f"   Default rates vary across states from {state_default.min():.1f}% to {state_default.max():.1f}%.")
print(f"   This suggests geographical location does affect loan default behavior.")

In [ ]:
# ============================================================
# 3.1.4 Feature Engineering - Term to numeric
# ============================================================
df_processed['term_numeric'] = df_processed['term'].apply(lambda x: int(str(x).strip().split()[0]))
print(f"Term numeric values: {df_processed['term_numeric'].unique()}")

In [ ]:
# ============================================================
# 3.1.5 Feature Engineering - Employment length to numeric
# ============================================================
def emp_length_to_num(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip()
    if '10+' in x:
        return 10
    elif '< 1' in x:
        return 0
    else:
        try:
            return int(x.split()[0])
        except:
            return np.nan

df_processed['emp_length_numeric'] = df_processed['emp_length'].apply(emp_length_to_num)
print(f"Employment length numeric values: {sorted(df_processed['emp_length_numeric'].dropna().unique())}")

In [ ]:
# ============================================================
# 3.1.6 Feature Engineering - Earliest credit line to years
# ============================================================
from datetime import datetime

def credit_line_years(x):
    if pd.isna(x):
        return np.nan
    try:
        date = pd.to_datetime(x, format='%b-%Y')
        return round((datetime(2020, 12, 1) - date).days / 365.25, 1)
    except:
        return np.nan

df_processed['credit_history_years'] = df_processed['earliest_cr_line'].apply(credit_line_years)
print(f"Credit history years - Mean: {df_processed['credit_history_years'].mean():.1f}, Median: {df_processed['credit_history_years'].median():.1f}")

In [ ]:
# ============================================================
# 3.1.7 Encode Target Variable
# ============================================================
df_processed['target'] = (df_processed['loan_status'] == 'Charged Off').astype(int)
print(f"Target encoding: Charged Off = 1, Fully Paid = 0")
print(f"Target distribution:\n{df_processed['target'].value_counts()}")

### 3.2 Missing Value Treatment

In [ ]:
# ============================================================
# 3.2.1 Handle Missing Values
# ============================================================
print("Missing values BEFORE treatment:")
print(df_processed.isnull().sum()[df_processed.isnull().sum() > 0])

# emp_title: Drop (too many unique values, not useful for modeling)
# emp_length: Fill with median
df_processed['emp_length_numeric'] = df_processed['emp_length_numeric'].fillna(df_processed['emp_length_numeric'].median())

# title: Drop (redundant with purpose)
# dti: Fill with median
df_processed['dti'] = df_processed['dti'].fillna(df_processed['dti'].median())

# revol_util: Fill with median
df_processed['revol_util'] = df_processed['revol_util'].fillna(df_processed['revol_util'].median())

# mort_acc: Fill with median based on total_acc groups
total_acc_avg = df_processed.groupby('total_acc')['mort_acc'].transform('median')
df_processed['mort_acc'] = df_processed['mort_acc'].fillna(total_acc_avg)
df_processed['mort_acc'] = df_processed['mort_acc'].fillna(df_processed['mort_acc'].median())

# pub_rec_bankruptcies: Fill with 0 (most common value)
df_processed['pub_rec_bankruptcies'] = df_processed['pub_rec_bankruptcies'].fillna(0)

# Update flags after filling
df_processed['mort_acc_flag'] = (df_processed['mort_acc'] > 0).astype(int)
df_processed['pub_rec_bankruptcies_flag'] = (df_processed['pub_rec_bankruptcies'] > 0).astype(int)

# Fill any remaining NaN values in ALL numeric columns
for col in df_processed.select_dtypes(include=['number']).columns:
    if df_processed[col].isnull().sum() > 0:
        df_processed[col] = df_processed[col].fillna(df_processed[col].median())

print("\nMissing values AFTER treatment:")
remaining_missing = df_processed.isnull().sum()[df_processed.isnull().sum() > 0]
print(remaining_missing if len(remaining_missing) > 0 else "No missing values in numerical features!")

### 3.3 Outlier Treatment

In [ ]:
# ============================================================
# 3.3.1 Outlier Treatment using IQR Capping
# ============================================================
def cap_outliers(df, col):
    Q1 = df[col].quantile(0.01)
    Q3 = df[col].quantile(0.99)
    df[col] = df[col].clip(lower=Q1, upper=Q3)
    return df

outlier_cols = ['annual_inc', 'revol_bal', 'open_acc', 'total_acc', 'dti']

print("Outlier treatment (1st-99th percentile capping):")
for col in outlier_cols:
    before_min, before_max = df_processed[col].min(), df_processed[col].max()
    df_processed = cap_outliers(df_processed, col)
    after_min, after_max = df_processed[col].min(), df_processed[col].max()
    print(f"  {col}: [{before_min:.0f}, {before_max:.0f}] -> [{after_min:.0f}, {after_max:.0f}]")

print("\n>> Insight: Outliers capped at 1st and 99th percentiles to reduce extreme values without losing too much information.")

### 3.4 Data Preparation for Modeling

In [ ]:
# ============================================================
# 3.4.1 Select Features for Modeling
# ============================================================

# Drop columns not needed for modeling
drop_cols = ['loan_status', 'emp_title', 'emp_length', 'title', 'issue_d', 
             'earliest_cr_line', 'Address', 'term', 'sub_grade', 'state']

# Features to keep
feature_cols = [col for col in df_processed.columns if col not in drop_cols + ['target']]
print(f"Features selected for modeling: {len(feature_cols)}")
print(f"Features: {feature_cols}")

In [ ]:
# ============================================================
# 3.4.2 Encode Categorical Variables
# ============================================================
# Identify remaining categorical columns
cat_features = df_processed[feature_cols].select_dtypes(include=['object']).columns.tolist()
num_features = df_processed[feature_cols].select_dtypes(include=['number']).columns.tolist()

print(f"Categorical features to encode: {cat_features}")
print(f"Numerical features: {num_features}")

# One-hot encode categorical variables
df_model = pd.get_dummies(df_processed[feature_cols + ['target']], columns=cat_features, drop_first=True)

# Ensure no NaN values remain (fill any residual NaN with 0)
df_model = df_model.fillna(0)

# Convert all columns to float to avoid dtype issues
for col in df_model.columns:
    if col != 'target':
        df_model[col] = df_model[col].astype(float)

print(f"\nDataset shape after encoding: {df_model.shape}")
print(f"Remaining NaN values: {df_model.isnull().sum().sum()}")

In [ ]:
# ============================================================
# 3.4.3 Train-Test Split
# ============================================================
X = df_model.drop('target', axis=1)
y = df_model['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\nTarget distribution in train: \n{y_train.value_counts(normalize=True).round(4) * 100}")
print(f"\nTarget distribution in test: \n{y_test.value_counts(normalize=True).round(4) * 100}")

In [ ]:
# ============================================================
# 3.4.4 Feature Scaling using StandardScaler
# ============================================================
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("Feature scaling applied using StandardScaler")
print(f"\nScaled Training data - Mean (should be ~0): {X_train_scaled.mean().mean():.6f}")
print(f"Scaled Training data - Std (should be ~1): {X_train_scaled.std().mean():.6f}")

---
## 4. Model Building

### 4.1 Logistic Regression Model (Scikit-Learn)

In [ ]:
# ============================================================
# 4.1.1 Build Logistic Regression Model
# ============================================================
lr_model = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced', solver='liblinear')
lr_model.fit(X_train_scaled, y_train)

print("Logistic Regression Model trained successfully!")
print(f"\nModel Parameters:")
print(f"  Solver: liblinear")
print(f"  Max iterations: 1000")
print(f"  Class weight: balanced (handles class imbalance)")
print(f"  Number of features: {X_train_scaled.shape[1]}")

In [ ]:
# ============================================================
# 4.1.2 Model Coefficients
# ============================================================
coefficients = pd.DataFrame({
    'Feature': X_train_scaled.columns,
    'Coefficient': lr_model.coef_[0],
    'Abs_Coefficient': np.abs(lr_model.coef_[0])
}).sort_values('Abs_Coefficient', ascending=False)

print("=" * 60)
print("MODEL COEFFICIENTS (sorted by absolute value)")
print("=" * 60)
print(coefficients[['Feature', 'Coefficient']].to_string(index=False))

# Visualize top 20 coefficients
top_coefs = coefficients.head(20)
fig, ax = plt.subplots(figsize=(12, 10))
colors = ['#e74c3c' if c > 0 else '#2ecc71' for c in top_coefs['Coefficient']]
ax.barh(top_coefs['Feature'], top_coefs['Coefficient'], color=colors, edgecolor='black')
ax.set_title('Top 20 Most Important Features (Logistic Regression Coefficients)', fontsize=16, fontweight='bold')
ax.set_xlabel('Coefficient Value')
ax.axvline(x=0, color='black', linewidth=1)
plt.tight_layout()
plt.savefig('../figures/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n>> ANSWER Q8: Features that heavily affected the outcome:")
for i, row in coefficients.head(10).iterrows():
    direction = 'increases' if row['Coefficient'] > 0 else 'decreases'
    print(f"   - {row['Feature']}: coefficient = {row['Coefficient']:.4f} ({direction} default probability)")

print(f"\nIntercept: {lr_model.intercept_[0]:.4f}")

In [ ]:
# ============================================================
# 4.1.3 Statsmodels Logistic Regression (for p-values)
# ============================================================
X_train_sm = sm.add_constant(X_train_scaled)
logit_model = sm.Logit(y_train, X_train_sm)
result = logit_model.fit(maxiter=1000, disp=0)

print("=" * 60)
print("STATSMODELS LOGISTIC REGRESSION SUMMARY")
print("=" * 60)
print(result.summary2())

---
## 5. Model Evaluation

### 5.1 Predictions

In [ ]:
# ============================================================
# 5.1.1 Generate Predictions
# ============================================================
y_pred = lr_model.predict(X_test_scaled)
y_pred_proba = lr_model.predict_proba(X_test_scaled)[:, 1]

print(f"Predictions generated for {len(y_test)} test samples")
print(f"\nPredicted class distribution: \n{pd.Series(y_pred).value_counts()}")
print(f"\nActual class distribution: \n{y_test.value_counts()}")

### 5.2 Classification Report & Confusion Matrix

In [ ]:
# ============================================================
# 5.2.1 Classification Report
# ============================================================
print("=" * 60)
print("CLASSIFICATION REPORT")
print("=" * 60)
print(classification_report(y_test, y_pred, target_names=['Fully Paid (0)', 'Charged Off (1)']))

print(f"\nOverall Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision (Charged Off): {precision_score(y_test, y_pred):.4f}")
print(f"Recall (Charged Off): {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score (Charged Off): {f1_score(y_test, y_pred):.4f}")

In [ ]:
# ============================================================
# 5.2.2 Confusion Matrix Visualization
# ============================================================
cm = confusion_matrix(y_test, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Raw counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Fully Paid', 'Charged Off'],
            yticklabels=['Fully Paid', 'Charged Off'])
axes[0].set_title('Confusion Matrix (Counts)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Percentages
cm_pct = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
sns.heatmap(cm_pct, annot=True, fmt='.1f', cmap='RdYlGn', ax=axes[1],
            xticklabels=['Fully Paid', 'Charged Off'],
            yticklabels=['Fully Paid', 'Charged Off'])
axes[1].set_title('Confusion Matrix (Percentages)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.suptitle('Confusion Matrix Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

tn, fp, fn, tp = cm.ravel()
print(f"\nTrue Negatives (Correctly predicted Fully Paid): {tn}")
print(f"False Positives (Incorrectly predicted Charged Off): {fp}")
print(f"False Negatives (Missed Defaults): {fn}")
print(f"True Positives (Correctly predicted Charged Off): {tp}")
print(f"\n>> Insight: The model correctly identifies {tp/(tp+fn)*100:.1f}% of actual defaults (Recall).")

### 5.3 ROC-AUC Curve

In [ ]:
# ============================================================
# 5.3.1 ROC-AUC Curve
# ============================================================
fpr, tpr, thresholds_roc = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(fpr, tpr, color='#e74c3c', linewidth=3, label=f'Logistic Regression (AUC = {roc_auc:.4f})')
ax.plot([0, 1], [0, 1], color='gray', linestyle='--', linewidth=2, label='Random Classifier (AUC = 0.5)')
ax.fill_between(fpr, tpr, alpha=0.1, color='#e74c3c')

# Find optimal threshold (Youden's J statistic)
j_scores = tpr - fpr
optimal_idx = np.argmax(j_scores)
optimal_threshold_roc = thresholds_roc[optimal_idx]
ax.scatter(fpr[optimal_idx], tpr[optimal_idx], marker='*', s=300, color='gold', 
           edgecolor='black', zorder=5, label=f'Optimal Threshold = {optimal_threshold_roc:.3f}')

ax.set_title('ROC-AUC Curve', fontsize=18, fontweight='bold')
ax.set_xlabel('False Positive Rate (1 - Specificity)', fontsize=14)
ax.set_ylabel('True Positive Rate (Sensitivity/Recall)', fontsize=14)
ax.legend(fontsize=13, loc='lower right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../figures/roc_auc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nROC-AUC Score: {roc_auc:.4f}")
print(f"Optimal Threshold (Youden's J): {optimal_threshold_roc:.4f}")
print(f"\n>> Insight: An AUC of {roc_auc:.4f} indicates the model has good discriminatory power.")
print("   The model can distinguish between defaulters and non-defaulters significantly better than random.")
print("   AUC > 0.7 is generally considered acceptable; > 0.8 is good; > 0.9 is excellent.")

### 5.4 Precision-Recall Curve

In [ ]:
# ============================================================
# 5.4.1 Precision-Recall Curve
# ============================================================
precision_curve, recall_curve, thresholds_pr = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall_curve, precision_curve)

fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(recall_curve, precision_curve, color='#3498db', linewidth=3, label=f'Logistic Regression (PR-AUC = {pr_auc:.4f})')
ax.fill_between(recall_curve, precision_curve, alpha=0.1, color='#3498db')

# Baseline
baseline = y_test.mean()
ax.axhline(y=baseline, color='gray', linestyle='--', linewidth=2, label=f'Baseline (Prevalence = {baseline:.3f})')

# Find optimal F1 threshold
f1_scores = 2 * (precision_curve * recall_curve) / (precision_curve + recall_curve + 1e-8)
optimal_f1_idx = np.argmax(f1_scores)
ax.scatter(recall_curve[optimal_f1_idx], precision_curve[optimal_f1_idx], 
           marker='*', s=300, color='gold', edgecolor='black', zorder=5,
           label=f'Best F1 Threshold = {thresholds_pr[optimal_f1_idx]:.3f}')

ax.set_title('Precision-Recall Curve', fontsize=18, fontweight='bold')
ax.set_xlabel('Recall (Sensitivity)', fontsize=14)
ax.set_ylabel('Precision', fontsize=14)
ax.legend(fontsize=12, loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
plt.tight_layout()
plt.savefig('../figures/precision_recall_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nPR-AUC Score: {pr_auc:.4f}")
print(f"Best F1 Score: {f1_scores[optimal_f1_idx]:.4f} at threshold {thresholds_pr[optimal_f1_idx]:.4f}")
print(f"\n>> Insight: The PR curve is particularly important for imbalanced datasets.")
print(f"   A PR-AUC of {pr_auc:.4f} shows the model's ability to identify defaulters while maintaining precision.")

### 5.5 Precision vs Recall Tradeoff Analysis

In [ ]:
# ============================================================
# 5.5.1 Precision-Recall Tradeoff at Different Thresholds
# ============================================================
thresholds_analysis = np.arange(0.1, 0.9, 0.05)

results = []
for thresh in thresholds_analysis:
    y_pred_thresh = (y_pred_proba >= thresh).astype(int)
    prec = precision_score(y_test, y_pred_thresh, zero_division=0)
    rec = recall_score(y_test, y_pred_thresh, zero_division=0)
    f1 = f1_score(y_test, y_pred_thresh, zero_division=0)
    acc = accuracy_score(y_test, y_pred_thresh)
    results.append({'Threshold': thresh, 'Precision': prec, 'Recall': rec, 'F1': f1, 'Accuracy': acc})

results_df = pd.DataFrame(results)

fig, ax = plt.subplots(figsize=(14, 8))
ax.plot(results_df['Threshold'], results_df['Precision'], 'b-o', linewidth=2, markersize=6, label='Precision')
ax.plot(results_df['Threshold'], results_df['Recall'], 'r-s', linewidth=2, markersize=6, label='Recall')
ax.plot(results_df['Threshold'], results_df['F1'], 'g-^', linewidth=2, markersize=6, label='F1-Score')
ax.plot(results_df['Threshold'], results_df['Accuracy'], 'm-d', linewidth=2, markersize=6, label='Accuracy')

# Highlight optimal F1 threshold
best_f1_idx = results_df['F1'].idxmax()
ax.axvline(x=results_df.loc[best_f1_idx, 'Threshold'], color='green', linestyle='--', alpha=0.5, 
           label=f'Best F1 Threshold = {results_df.loc[best_f1_idx, "Threshold"]:.2f}')

ax.set_title('Precision-Recall-F1-Accuracy Tradeoff at Different Thresholds', fontsize=16, fontweight='bold')
ax.set_xlabel('Classification Threshold', fontsize=14)
ax.set_ylabel('Score', fontsize=14)
ax.legend(fontsize=12, loc='center left')
ax.grid(True, alpha=0.3)
ax.set_xlim([0.1, 0.85])
plt.tight_layout()
plt.savefig('../figures/precision_recall_tradeoff.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nThreshold Analysis Table:")
print(results_df.round(4).to_string(index=False))

---
## 6. Tradeoff Questions & Business Analysis

In [ ]:
# ============================================================
# 6.1 Tradeoff Question 1: Detecting Real Defaulters with Less False Positives
# ============================================================
print("=" * 80)
print("TRADEOFF QUESTION 1: Detecting Real Defaulters with Less False Positives")
print("=" * 80)
print("""
BUSINESS CONTEXT:
- False Positives (FP): Predicting a good borrower as defaulter -> LOST BUSINESS OPPORTUNITY
- We want to MAXIMIZE RECALL (catch real defaulters) while keeping FP low

STRATEGY: Focus on RECALL with acceptable Precision

APPROACH:
1. Lower the classification threshold to catch more defaults
2. Use class_weight='balanced' (already implemented)
3. Monitor the precision-recall tradeoff carefully

If we want to detect real defaulters while minimizing false positives, we should:
- Use a MODERATE threshold that balances both metrics
- Focus on F1-Score as the primary metric (harmonic mean of precision & recall)
- Consider the COST of each type of error:
  * Cost of FP (rejecting good borrower) = Lost interest income
  * Cost of FN (approving bad borrower) = Loan amount loss
""")

# Scenario 1: High Recall threshold (conservative)
high_recall_thresh = 0.3
y_pred_hr = (y_pred_proba >= high_recall_thresh).astype(int)
print(f"\n--- Scenario: Lower Threshold ({high_recall_thresh}) for Higher Recall ---")
print(f"Precision: {precision_score(y_test, y_pred_hr):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_hr):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_hr):.4f}")
print(f"False Positives: {((y_pred_hr == 1) & (y_test == 0)).sum()}")
print(f"False Negatives: {((y_pred_hr == 0) & (y_test == 1)).sum()}")

print(f"\n>> ANSWER Q6: From a bank's perspective, the primary focus should be on RECALL.")
print("   Reason: Missing a defaulter (False Negative) is MORE COSTLY than rejecting a")
print("   good borrower (False Positive). The cost of NPA far exceeds the lost interest income.")
print("   However, if the bank wants balanced growth + safety, F1-Score is preferred.")

In [ ]:
# ============================================================
# 6.2 Tradeoff Question 2: NPA Prevention - Play it Safe
# ============================================================
print("=" * 80)
print("TRADEOFF QUESTION 2: NPA Prevention - Playing it Safe")
print("=" * 80)
print("""
BUSINESS CONTEXT:
- NPA (Non-Performing Asset) is a critical risk for banks
- We should NOT disburse loans to potential defaulters
- Priority: MAXIMIZE RECALL (catch ALL defaulters, even at the cost of some FP)

STRATEGY: Use a LOWER threshold to flag more potential defaults
""")

# Conservative approach - very low threshold
safe_thresh = 0.25
y_pred_safe = (y_pred_proba >= safe_thresh).astype(int)

print(f"--- Conservative Approach: Threshold = {safe_thresh} ---")
print(f"Classification Report:")
print(classification_report(y_test, y_pred_safe, target_names=['Fully Paid', 'Charged Off']))

cm_safe = confusion_matrix(y_test, y_pred_safe)
tn_s, fp_s, fn_s, tp_s = cm_safe.ravel()

print(f"\nWith threshold = {safe_thresh}:")
print(f"  Defaulters caught: {tp_s} out of {tp_s + fn_s} ({tp_s/(tp_s+fn_s)*100:.1f}%)")
print(f"  Good borrowers incorrectly flagged: {fp_s} out of {tn_s + fp_s} ({fp_s/(tn_s+fp_s)*100:.1f}%)")
print(f"  Missed defaulters (NPA risk): {fn_s}")

# Cost-benefit analysis
avg_loan = df['loan_amnt'].mean()
avg_interest = df['int_rate'].mean()
print(f"\n--- Cost-Benefit Analysis ---")
print(f"  Average loan amount: ${avg_loan:,.0f}")
print(f"  Average interest rate: {avg_interest:.1f}%")
print(f"  Estimated NPA savings (caught defaults): ${tp_s * avg_loan:,.0f}")
print(f"  Estimated lost opportunity (false positives): ${fp_s * avg_loan * avg_interest/100:,.0f}")
print(f"  Net benefit: ${tp_s * avg_loan - fp_s * avg_loan * avg_interest/100:,.0f}")

print("\n>> Recommendation: Use conservative threshold (0.25-0.30) to minimize NPA risk.")
print("   The cost of a bad loan far exceeds the opportunity cost of a missed good loan.")

In [ ]:
# ============================================================
# 6.3 Visualization: Impact of Different Thresholds
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(22, 7))

threshold_scenarios = [
    (0.25, 'Conservative (NPA Safe)', '#e74c3c'),
    (0.5, 'Default Threshold', '#3498db'),
    (0.65, 'Aggressive (Growth)', '#2ecc71')
]

for idx, (thresh, title, color) in enumerate(threshold_scenarios):
    y_p = (y_pred_proba >= thresh).astype(int)
    cm_t = confusion_matrix(y_test, y_p)
    sns.heatmap(cm_t, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['Fully Paid', 'Charged Off'],
                yticklabels=['Fully Paid', 'Charged Off'])
    prec = precision_score(y_test, y_p, zero_division=0)
    rec = recall_score(y_test, y_p, zero_division=0)
    axes[idx].set_title(f'{title}\nThreshold={thresh} | P={prec:.2f} | R={rec:.2f}', 
                        fontsize=13, fontweight='bold', color=color)
    axes[idx].set_xlabel('Predicted')
    axes[idx].set_ylabel('Actual')

plt.suptitle('Confusion Matrices at Different Thresholds', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/threshold_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n>> ANSWER Q7: Gap in Precision and Recall affects the bank as follows:")
print("   - High Precision, Low Recall: Bank misses many defaulters -> HIGH NPA RISK")
print("   - High Recall, Low Precision: Bank rejects many good borrowers -> LOST REVENUE")
print("   - The gap indicates the model struggles to simultaneously identify all defaults")
print("     while maintaining accuracy in predictions.")
print("   - A large gap forces the bank to choose: safety (recall) vs growth (precision)")
print("   - For loan defaults, RECALL should be prioritized as NPA cost >> opportunity cost")

---
## 7. Actionable Insights & Recommendations

In [ ]:
# ============================================================
# 7.1 Summary of All Findings
# ============================================================
print("=" * 80)
print("               ACTIONABLE INSIGHTS & RECOMMENDATIONS")
print("               Author: Vidyasagar | Data Scientist")
print("=" * 80)

fully_paid_pct = (df['loan_status'] == 'Fully Paid').mean() * 100
top2_titles = df['emp_title'].value_counts().head(2).index.tolist()
majority_home = df['home_ownership'].value_counts().index[0]

print(f"""
============================================================
QUESTIONNAIRE ANSWERS
============================================================

Q1. What percentage of customers have fully paid their Loan Amount?
    ANSWER: {fully_paid_pct:.2f}% of customers have fully paid their loan.

Q2. Comment about the correlation between Loan Amount and Installment features.
    ANSWER: There is a very strong positive correlation ({df['loan_amnt'].corr(df['installment']):.4f})
    between Loan Amount and Installment. This is expected because the monthly installment
    is directly calculated from the loan amount, interest rate, and term. This indicates
    potential multicollinearity, and one of these features could be dropped during modeling.

Q3. The majority of people have home ownership as _______.
    ANSWER: The majority of people have home ownership as '{majority_home}'.

Q4. People with grades 'A' are more likely to fully pay their loan. (T/F)
    ANSWER: TRUE. Grade A borrowers have the highest repayment rate among all grades,
    with significantly lower default rates compared to grades D, E, F, and G.

Q5. Name the top 2 afforded job titles.
    ANSWER: The top 2 job titles are '{top2_titles[0]}' and '{top2_titles[1]}'.

Q6. Thinking from a bank's perspective, which metric should our primary focus be on?
    ANSWER: RECALL should be the primary focus. In the lending business, the cost of
    approving a defaulter (False Negative) far exceeds the cost of rejecting a good
    borrower (False Positive). NPA directly impacts the bank's balance sheet, while
    a rejected good borrower is merely a missed opportunity. However, for a balanced
    approach, F1-Score can be used as it considers both precision and recall.

Q7. How does the gap in precision and recall affect the bank?
    ANSWER: The precision-recall gap forces the bank into a strategic tradeoff:
    - If precision >> recall: Many defaulters are missed, leading to HIGH NPA.
    - If recall >> precision: Many good borrowers rejected, leading to LOST REVENUE.
    The bank must choose based on its risk appetite. A conservative bank should
    prioritize recall; a growth-oriented bank may tolerate more risk for higher revenue.

Q8. Which were the features that heavily affected the outcome?
    ANSWER: The top features affecting loan default prediction are:""")

for i, row in coefficients.head(5).iterrows():
    direction = 'INCREASES' if row['Coefficient'] > 0 else 'DECREASES'
    print(f"    - {row['Feature']} (coef: {row['Coefficient']:.4f}) -> {direction} default risk")

print(f"""
Q9. Will the results be affected by geographical location? (Yes/No)
    ANSWER: YES. Analysis shows default rates vary significantly across states.
    Economic conditions, employment rates, and cost of living differ by region,
    all of which influence a borrower's ability to repay loans.

============================================================
KEY BUSINESS INSIGHTS
============================================================

1. RISK SEGMENTATION BY GRADE:
   - Grade A-B borrowers have <15% default rate -> LOW RISK -> Favorable terms
   - Grade D-G borrowers have >30% default rate -> HIGH RISK -> Stricter screening

2. TERM EFFECT:
   - 60-month loans have significantly higher default rates than 36-month
   - Recommendation: Incentivize shorter loan terms

3. INTEREST RATE PARADOX:
   - Higher interest rates correlate with higher defaults
   - Risk-based pricing must be carefully calibrated

4. EMPLOYMENT STABILITY:
   - Borrowers with 10+ years employment have lower default rates
   - Employment length is a strong predictor of repayment capability

5. DTI RATIO THRESHOLD:
   - Borrowers with DTI > 25 show significantly higher default rates
   - Consider DTI caps for loan approval

============================================================
RECOMMENDATIONS
============================================================

1. IMPLEMENT TIERED APPROVAL:
   - Automatic approval for low-risk (Grade A-B, low DTI, stable employment)
   - Enhanced review for medium-risk (Grade C-D)
   - Additional collateral/guarantor for high-risk (Grade E-G)

2. THRESHOLD OPTIMIZATION:
   - Use threshold of 0.25-0.30 for conservative lending (NPA minimization)
   - Use threshold of 0.45-0.50 for balanced growth

3. FEATURE MONITORING:
   - Continuously monitor top predictive features
   - Implement early warning system based on DTI changes

4. GEOGRAPHIC RISK ADJUSTMENT:
   - Apply state-level risk factors to loan pricing
   - Monitor regional economic indicators

5. MODEL IMPROVEMENT:
   - Consider ensemble methods (Random Forest, XGBoost) for better performance
   - Implement SMOTE or other oversampling techniques for class imbalance
   - Regular model retraining with updated data

ROC-AUC Score: {roc_auc:.4f}
The model demonstrates strong discriminatory power and can be deployed
as part of the loan approval pipeline with appropriate threshold tuning.
""")

In [ ]:
# ============================================================
# 7.2 Final Summary Visualization
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# 1. Default Rate by Grade
grade_default = df.groupby('grade')['loan_status'].apply(lambda x: (x == 'Charged Off').mean() * 100)
grade_default.plot(kind='bar', ax=axes[0, 0], color=sns.color_palette('RdYlGn_r', 7), edgecolor='black')
axes[0, 0].set_title('Default Rate by Grade', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Default Rate (%)')
axes[0, 0].tick_params(axis='x', rotation=0)
for p in axes[0, 0].patches:
    axes[0, 0].annotate(f'{p.get_height():.1f}%', 
                        (p.get_x() + p.get_width()/2., p.get_height()),
                        ha='center', va='bottom', fontweight='bold')

# 2. Default Rate by Term
term_default = df.groupby('term')['loan_status'].apply(lambda x: (x == 'Charged Off').mean() * 100)
term_default.plot(kind='bar', ax=axes[0, 1], color=['#2ecc71', '#e74c3c'], edgecolor='black')
axes[0, 1].set_title('Default Rate by Term', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Default Rate (%)')
axes[0, 1].tick_params(axis='x', rotation=0)
for p in axes[0, 1].patches:
    axes[0, 1].annotate(f'{p.get_height():.1f}%', 
                        (p.get_x() + p.get_width()/2., p.get_height()),
                        ha='center', va='bottom', fontweight='bold')

# 3. ROC Curve
axes[1, 0].plot(fpr, tpr, color='#e74c3c', linewidth=3, label=f'AUC = {roc_auc:.4f}')
axes[1, 0].plot([0, 1], [0, 1], color='gray', linestyle='--')
axes[1, 0].fill_between(fpr, tpr, alpha=0.1, color='#e74c3c')
axes[1, 0].set_title('ROC-AUC Curve', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('False Positive Rate')
axes[1, 0].set_ylabel('True Positive Rate')
axes[1, 0].legend(fontsize=12)

# 4. Feature Importance (Top 10)
top10 = coefficients.head(10)
colors = ['#e74c3c' if c > 0 else '#2ecc71' for c in top10['Coefficient']]
axes[1, 1].barh(top10['Feature'], top10['Coefficient'], color=colors, edgecolor='black')
axes[1, 1].set_title('Top 10 Feature Importance', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Coefficient')
axes[1, 1].axvline(x=0, color='black', linewidth=1)

plt.suptitle('Loan Default Prediction - Executive Summary\nby Vidyasagar, Data Scientist', 
             fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../figures/executive_summary.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "=" * 60)
print("All figures saved to ../figures/ directory")
print("=" * 60)

In [ ]:
# ============================================================
# Final Model Performance Summary
# ============================================================
print("\n" + "=" * 60)
print("         FINAL MODEL PERFORMANCE SUMMARY")
print("=" * 60)
print(f"""  
  Model: Logistic Regression (class_weight='balanced')
  
  Training Samples: {X_train.shape[0]}
  Test Samples: {X_test.shape[0]}
  Number of Features: {X_train.shape[1]}
  
  ---- Default Threshold (0.5) ----
  Accuracy:  {accuracy_score(y_test, y_pred):.4f}
  Precision: {precision_score(y_test, y_pred):.4f}
  Recall:    {recall_score(y_test, y_pred):.4f}
  F1-Score:  {f1_score(y_test, y_pred):.4f}
  ROC-AUC:   {roc_auc:.4f}
  PR-AUC:    {pr_auc:.4f}
  
  ---- Recommended Threshold (0.30) ----
  Precision: {precision_score(y_test, (y_pred_proba >= 0.30).astype(int)):.4f}
  Recall:    {recall_score(y_test, (y_pred_proba >= 0.30).astype(int)):.4f}
  F1-Score:  {f1_score(y_test, (y_pred_proba >= 0.30).astype(int)):.4f}
  
  Conclusion: The model performs well for identifying loan defaults.
  Use threshold tuning based on business risk appetite.
""")